In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# --- Paths ---
csv_path = "/content/drive/MyDrive/miniMIAS_Dataset/mias_info_with_yolo.csv"    # path to your CSV
output_folder = "/content/drive/MyDrive/MIAS/Labels"  # where YOLO txt files will be saved

# Create output folder if it doesn’t exist
os.makedirs(output_folder, exist_ok=True)

# --- Load CSV ---
df = pd.read_csv(csv_path)

# --- Group by ID ---
for img_id, group in df.groupby("ID"):
    # Collect YOLO bbox rows (ignore NaN or empty values)
    bboxes = group["yolo_bbox"].dropna().tolist()

    if len(bboxes) == 0:
        # no tumor → skip creating file
        continue

    lines = []
    for bbox in bboxes:
        # bbox format in CSV: "(0, 0.52, 0.58, 0.38, 0.38)"
        # Remove parentheses and split
        bbox = bbox.strip("()")
        parts = bbox.split(",")
        parts = [p.strip() for p in parts]

        if len(parts) != 5:
            continue  # skip malformed row

        class_id, x_c, y_c, w, h = parts
        line = f"{class_id} {x_c} {y_c} {w} {h}"
        lines.append(line)

    # Save YOLO label file
    txt_path = os.path.join(output_folder, f"{img_id}.txt")
    with open(txt_path, "w") as f:
        f.write("\n".join(lines))

print("✅ YOLO .txt label files created in:", output_folder)